In [1]:
!pip install torch torchvision pillow numpy

In [10]:
import os
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
from stl import mesh  # Для загрузки STL-файлов
from torch_geometric.data import Data
from torch_geometric.nn import global_max_pool
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Блоки векторизации изображений

In [2]:
input_dir = "train/train_data/images"  # папка с исходными изображениями
output_dir = "train/images/image_vector"  # папка для сохранения векторов
os.makedirs(output_dir, exist_ok=True)

# Загрузка модели ResNet-50
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*(list(model.children())[:-1]))  # удаляем последний слой
model.eval()

# Трансформации для изображения
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.Lambda(lambda x: x.convert('RGB') if x.mode != 'RGB' else x),  # конвертируем в RGB если нужно
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# Перебор всех изображений в папке
for filename in os.listdir(input_dir):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # проверяем расширение
        try:
            # Полный путь к изображению
            image_path = os.path.join(input_dir, filename)
            
            # Загрузка и преобразование изображения
            image = Image.open(image_path)
            image_tensor = transform(image).unsqueeze(0)
            
            # Извлечение признаков
            with torch.no_grad():
                features = model(image_tensor)
                features = features.squeeze().numpy()  # сразу в numpy

            # L2-нормализация
            features = features / np.linalg.norm(features)
            
            # Формируем имя выходного файла (без расширения + '_image_vector.npy')
            base_name = os.path.splitext(filename)[0]
            output_filename = f"{base_name}_image_vector.npy"
            output_path = os.path.join(output_dir, output_filename)
            
            # Сохранение вектора
            np.save(output_path, features)
            if '00_0' in filename:
                print(f"Обработано: {filename} -> {output_filename}")
            
        except Exception as e:
            print(f"Ошибка при обработке {filename}: {str(e)}")

print("Обработка всех изображений завершена!")

Обработано: 0000_0.png -> 0000_0_image_vector.npy
Обработано: 0100_0.png -> 0100_0_image_vector.npy
Обработано: 0200_0.png -> 0200_0_image_vector.npy
Обработано: 0300_0.png -> 0300_0_image_vector.npy
Обработано: 0400_0.png -> 0400_0_image_vector.npy
Обработано: 0500_0.png -> 0500_0_image_vector.npy
Обработка всех изображений завершена!


In [ ]:
# Блоки векторизации STL

In [13]:
input_dir = "train/train_data/models"  # Папка с STL-файлами
output_dir = "train/images/model_vector"  # Папка для сохранения векторов
os.makedirs(output_dir, exist_ok=True)

# Упрощенная реализация PointNet (без трансформационных сетей)
class PointNet(torch.nn.Module):
    def __init__(self, out_dim=2048):
        super().__init__()
        self.conv1 = torch.nn.Conv1d(3, 64, 1)
        self.conv2 = torch.nn.Conv1d(64, 128, 1)
        self.conv3 = torch.nn.Conv1d(128, out_dim, 1)  # Фиксируем out_dim
        self.bn1 = torch.nn.BatchNorm1d(64)
        self.bn2 = torch.nn.BatchNorm1d(128)
        self.bn3 = torch.nn.BatchNorm1d(out_dim)
        
    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = torch.relu(self.bn2(self.conv2(x)))
        x = self.bn3(self.conv3(x))  # Размер: (batch_size, 2048, num_points)
        return torch.max(x, dim=-1)[0]  # Глобальный макс-пулинг -> (batch_size, 2048)

def stl_to_point_cloud(stl_path, num_points=1024):
    """Преобразование STL в облако точек с нормализацией"""
    stl_mesh = mesh.Mesh.from_file(stl_path)
    points = stl_mesh.vectors.reshape(-1, 3)
    if len(points) > num_points:
        points = points[np.random.choice(len(points), num_points, replace=False)]
    points -= np.mean(points, axis=0)
    points /= np.max(np.abs(points))
    return torch.tensor(points, dtype=torch.float32).T  # (3, N)

# Инициализация модели
model = PointNet(out_dim=2048)
model.eval()

def stl_to_point_cloud(stl_path, num_points=1024):
    """Загрузка STL и преобразование в облако точек"""
    stl_mesh = mesh.Mesh.from_file(stl_path)
    points = stl_mesh.vectors.reshape(-1, 3)  # Все вершины треугольников
    
    # Если точек слишком много, делаем случайную выборку
    if len(points) > num_points:
        indices = np.random.choice(len(points), num_points, replace=False)
        points = points[indices]
    
    # Нормализация в единичный куб
    points -= np.mean(points, axis=0)
    points /= np.max(np.abs(points))
    return torch.tensor(points, dtype=torch.float32).T  # (3, N)

In [14]:
# Обработка всех STL-файлов
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.stl'):
        try:
            stl_path = os.path.join(input_dir, filename)
            
            # Преобразование STL в облако точек
            point_cloud = stl_to_point_cloud(stl_path)
            
            # Извлечение признаков
            with torch.no_grad():
                features = model(point_cloud.unsqueeze(0))  # Добавляем batch-размер
                features = features.squeeze().numpy()
            
            # L2-нормализация
            features = features / np.linalg.norm(features)
            
            # Сохранение вектора
            base_name = os.path.splitext(filename)[0]
            output_path = os.path.join(output_dir, f"{base_name}_model_vector.npy")
            np.save(output_path, features)
            
            if '00_0' in filename:
                print(f"Обработано: {filename} -> {base_name}_model_vector.npy")
                
        except Exception as e:
            print(f"Ошибка при обработке {filename}: {str(e)}")

print("Обработка всех STL-файлов завершена!")

Обработка всех STL-файлов завершена!


In [ ]:
# Блок извлечения векторов из папок

In [16]:
# Указываем путь к папке с векторами
folder_path = "train/images/model_vector"  # замените на реальный путь

# Собираем все файлы .npy в папке
npy_files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

# Загружаем векторы в список
model_vectors = []
for file in npy_files:
    vector = np.load(os.path.join(folder_path, file))
    model_vectors.append(vector)


folder_path = "train/images/image_vector"  # замените на реальный путь

# Собираем все файлы .npy в папке
npy_files = [f for f in os.listdir(folder_path) if f.endswith('.npy')]

# Загружаем векторы в список
image_vectors = []
for file in npy_files:
    vector = np.load(os.path.join(folder_path, file))
    image_vectors.append(vector)

#print([len(vec) for vec in model_vectors])  # Проверить длины model_vectors
#print([len(vec) for vec in image_vectors])  # Проверить длины image_vectors
# Преобразуем список в numpy-массив для удобства (если векторы одинаковой длины)


In [17]:
model_vectors_array = np.array(model_vectors)
image_vectors_array = np.array(image_vectors)

In [ ]:
# Блоки поиска ближайших векторов.

In [18]:
query_vector = model_vectors_array[0].reshape(1, -1)  # Пример: ищем похожие на первый вектор
similarities = cosine_similarity(query_vector, image_vectors_array).flatten()

top5_indices = np.argsort(similarities)[::-1][1:6]
top5_vectors = image_vectors_array[top5_indices]
top5_files = [npy_files[i] for i in top5_indices]

print("Топ-5 ближайших векторов:")
for vec, file, sim in zip(top5_vectors, top5_files, similarities[top5_indices]):
    print(f"{file}: схожесть = {sim:.4f}\n{vec}\n")

Топ-5 ближайших векторов:
0341_4_image_vector.npy: схожесть = 0.5105
[0.02610345 0.0144286  0.01983626 ... 0.00265987 0.01200935 0.00739379]

0178_20_image_vector.npy: схожесть = 0.5087
[0.02269515 0.02856544 0.02481004 ... 0.00512752 0.00257433 0.00167598]

0275_4_image_vector.npy: схожесть = 0.5086
[0.03993442 0.00366707 0.00246136 ... 0.00545141 0.01913376 0.02416422]

0340_10_image_vector.npy: схожесть = 0.5081
[0.02199505 0.01686841 0.00882186 ... 0.00675416 0.02264574 0.00949392]

0043_20_image_vector.npy: схожесть = 0.5077
[0.02210515 0.0216442  0.01470946 ... 0.00113921 0.00097669 0.0069649 ]



In [19]:
query_vector = image_vectors_array[0].reshape(1, -1)  # Пример: ищем похожие на первый вектор
similarities = cosine_similarity(query_vector, model_vectors_array).flatten()

top5_indices = np.argsort(similarities)[::-1][1:6]
top5_vectors = model_vectors_array[top5_indices]
top5_files = [npy_files[i] for i in top5_indices]

print("Топ-5 ближайших векторов:")
for vec, file, sim in zip(top5_vectors, top5_files, similarities[top5_indices]):
    print(f"{file}: схожесть = {sim:.4f}\n{vec}\n")

Топ-5 ближайших векторов:
0017_1_image_vector.npy: схожесть = 0.4648
[0.04808094 0.01988083 0.01407408 ... 0.02472101 0.00159843 0.0276232 ]

0002_14_image_vector.npy: схожесть = 0.4619
[0.04757209 0.01911085 0.01289003 ... 0.0243462  0.00233077 0.0234149 ]

0019_13_image_vector.npy: схожесть = 0.4594
[0.04679163 0.01904525 0.01424565 ... 0.02393358 0.00225289 0.02219982]

0003_25_image_vector.npy: схожесть = 0.4567
[0.04848671 0.01951078 0.01529248 ... 0.02446476 0.00186564 0.02586157]

0001_9_image_vector.npy: схожесть = 0.4566
[ 0.04822005  0.02021776  0.01590581 ...  0.02482694 -0.00164084
  0.027797  ]



In [24]:
query_vector = image_vectors_array[0].reshape(1, -1)  # Пример: ищем похожие на первый вектор
similarities = cosine_similarity(query_vector, image_vectors_array).flatten()

top5_vectors = similarities[:26]
top5_files = [npy_files[i] for i in range(0, 26)]

print("Топ-5 ближайших векторов:")
for vec, file, sim in zip(top5_vectors, top5_files, similarities[:26]):
    print(f"{file}: схожесть = {sim:.4f}\n{vec}\n")

Топ-5 ближайших векторов:
0000_0_image_vector.npy: схожесть = 1.0000
1.0

0000_10_image_vector.npy: схожесть = 0.6259
0.6258577704429626

0000_11_image_vector.npy: схожесть = 0.5882
0.588249921798706

0000_12_image_vector.npy: схожесть = 0.9459
0.9459471702575684

0000_13_image_vector.npy: схожесть = 0.7868
0.7867915630340576

0000_14_image_vector.npy: схожесть = 0.6439
0.6438932418823242

0000_15_image_vector.npy: схожесть = 0.7883
0.7883498668670654

0000_16_image_vector.npy: схожесть = 0.6156
0.6156439781188965

0000_17_image_vector.npy: схожесть = 0.6164
0.6164241433143616

0000_18_image_vector.npy: схожесть = 0.8814
0.8813884258270264

0000_19_image_vector.npy: схожесть = 0.9047
0.9046547412872314

0000_1_image_vector.npy: схожесть = 0.8529
0.8528616428375244

0000_20_image_vector.npy: схожесть = 0.6263
0.6263169050216675

0000_21_image_vector.npy: схожесть = 0.7837
0.7837072610855103

0000_22_image_vector.npy: схожесть = 0.6876
0.6876150369644165

0000_23_image_vector.npy: схожес